# Some potential audiences are:

1. Homeowners who want to increase the sale price of their homes through home improvement projects
2. Advocacy groups who want to promote affordable housing
3. Local elected officials who want to understand how their policy ideas (e.g. zoning changes, permitting) might impact home prices
4. Real estate investors looking for potential "fixer-uppers" or "tear-downs"

# Three things to be sure you establish during this phase are:

1. **Objectives:** what questions are you trying to answer, and for whom?
2. **Project plan:** you may want to establish more formal project management practices, such as daily stand-ups or using a Trello board, to plan the time you have remaining. Regardless, you should determine the division of labor, communication expectations, and timeline.
3. **Success criteria:** what does a successful project look like? How will you know when you have achieved it?

# READ THIS: Import the following data files from https://info.kingcounty.gov/assessor/DataDownload/default.aspx
## Download the files to local repo data directory
> 1) Real Property Sales (.ZIP, csv) <BR>
> 2) Parcel (.ZIP, csv) <BR>
> 3) Residential Building (.ZIP, csv) <BR>
> 4) Unit Breakdown (.ZIP)<BR>


In [39]:
import os
import sys

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.diagnostic import linear_rainbow, het_breuschpagan
from statsmodels.stats.outliers_influence import variance_inflation_factor

from eda.eda import *
# from lr_model.build_lr import *

ModuleNotFoundError: No module named 'lr_model.build_lr'

### Andrew's scratchwork below:
____

In [59]:
df_merged = consolidate_data(year=2019, create=True)

KeyError: 'Merged_Key'

In [56]:
cols = list(df_merged.columns)
cols = cols[2:4]  + cols[7:8] + cols[10:11] + cols[14:18] + cols[27:30] + cols[38:41] + cols[42:43] + cols[44:47] 
df = df_merged[cols] 

In [57]:
df = df_merged.copy()
df = df.dropna().reset_index(drop = True)
# df.drop(columns = ['DocumentDate', 'DistrictName', 'Address', 'Merged_Key', 'PropertyType'], inplace = True)
print(df.shape)
df.head()


(3884, 60)


,Merged_Key,DocumentDate,SalePrice,PropertyType,PrincipalUse,PropertyClass,PropType,Area,SubArea,DistrictName,...,SqFtDeck,HeatSystem,Bedrooms,BathHalfCount,Bath3qtrCount,BathFullCount,FpSingleStory,FpMultiStory,YrRenovated,PcntComplete
0,98400000450,2019,409950,11,6,8,R,51.0,6.0,KING COUNTY,...,140.0,5.0,3.0,1.0,0.0,2.0,0.0,1.0,0.0,0.0
1,797320002320,2019,540000,3,6,8,R,23.0,4.0,KING COUNTY,...,0.0,5.0,3.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
2,82607009096,2019,930000,11,6,8,R,70.0,3.0,KING COUNTY,...,0.0,5.0,3.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0
3,410200000075,2019,379950,11,6,8,R,40.0,9.0,KING COUNTY,...,520.0,4.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,868229001120,2019,620000,14,6,8,R,95.0,10.0,KING COUNTY,...,0.0,5.0,3.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3884 entries, 0 to 3883
Data columns (total 60 columns):
Merged_Key                3884 non-null int64
DocumentDate              3884 non-null int64
SalePrice                 3884 non-null int64
PropertyType              3884 non-null int64
PrincipalUse              3884 non-null int64
PropertyClass             3884 non-null int64
PropType                  3884 non-null object
Area                      3884 non-null float64
SubArea                   3884 non-null float64
DistrictName              3884 non-null object
SqFtLot                   3884 non-null float64
WaterSystem               3884 non-null float64
SewerSystem               3884 non-null float64
Access                    3884 non-null float64
SeattleSkyline            3884 non-null float64
LakeWashington            3884 non-null float64
LakeSammamish             3884 non-null float64
SmallLakeRiverCreek       3884 non-null float64
OtherView                 3884 non-null flo

In [15]:
df.corr()

,Merged_Key,DocumentDate,SalePrice,PropertyType,PrincipalUse,PropertyClass,Area,SubArea,SqFtLot,WaterSystem,...,SqFtDeck,HeatSystem,Bedrooms,BathHalfCount,Bath3qtrCount,BathFullCount,FpSingleStory,FpMultiStory,YrRenovated,PcntComplete
Merged_Key,1.000000,NaN,-0.022253,0.040551,-0.013608,0.025131,0.006339,0.095817,-0.137731,0.257468,...,-0.044945,0.066358,0.043591,0.008387,-0.033282,0.048213,0.032982,-0.014389,-0.031693,-0.017287
DocumentDate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SalePrice,-0.022253,NaN,1.000000,-0.005165,0.015982,-0.060807,0.082551,0.009070,-0.066596,-0.009903,...,0.074741,0.080091,0.106218,0.113627,0.036762,0.175239,0.073894,0.088082,-0.005312,-0.013907
PropertyType,0.040551,NaN,-0.005165,1.000000,-0.018424,-0.060156,-0.050087,0.005188,-0.146624,0.094008,...,0.000189,0.087836,0.092237,0.038385,0.012373,0.059078,0.019104,0.006497,-0.000157,0.332727
PrincipalUse,-0.013608,NaN,0.015982,-0.018424,1.000000,-0.333725,-0.016969,0.028120,-0.003165,0.011230,...,-0.015732,0.011709,-0.018247,0.001389,-0.018158,-0.016120,0.012974,-0.013239,-0.005288,-0.000785
PropertyClass,0.025131,NaN,-0.060807,-0.060156,-0.333725,1.000000,0.001053,0.035990,0.005296,0.007609,...,0.009775,-0.017378,0.011313,0.030645,-0.029856,0.012483,-0.010030,-0.000211,0.017821,-0.168370
Area,0.006339,NaN,0.082551,-0.050087,-0.016969,0.001053,1.000000,0.043277,0.172436,-0.348267,...,0.123736,-0.125731,-0.196714,0.071476,-0.043885,0.055523,0.037990,0.031162,0.058255,0.019493
SubArea,0.095817,NaN,0.009070,0.005188,0.028120,0.035990,0.043277,1.000000,-0.051866,0.100274,...,-0.019356,0.031184,0.038330,0.101288,-0.044598,0.086381,0.064035,0.011829,-0.018884,-0.026043
SqFtLot,-0.137731,NaN,-0.066596,-0.146624,-0.003165,0.005296,0.172436,-0.051866,1.000000,-0.515045,...,-0.081998,-0.484187,-0.432102,-0.109309,-0.103091,-0.265125,-0.004351,-0.074650,-0.024350,-0.003923
WaterSystem,0.257468,NaN,-0.009903,0.094008,0.011230,0.007609,-0.348267,0.100274,-0.515045,1.000000,...,-0.080622,0.271566,0.300557,0.091833,0.039588,0.161226,0.087418,0.078535,-0.060439,-0.031428


In [16]:
col = []
dic = df_merged.corr()['SalePrice'].to_dict()
for x in dic:
    if dic[x] >= abs(0.10):
        col.append(x)
        print(x)

SalePrice
NbrLivingUnits
Stories
SqFt1stFloor
SqFt2ndFloor
SqFtTotLiving
SqFtGarageAttached
SqFtOpenPorch
Bedrooms
BathFullCount


In [17]:
# # create a smaller df to save space and processing power
fsm_df = df_merged[col].copy()
fsm_df.dropna(inplace=True)
fsm = ols(formula="SalePrice ~ NbrLivingUnits + Stories + SqFt1stFloor + SqFt2ndFloor + SqFtTotLiving + SqFtGarageAttached + SqFtOpenPorch + Bedrooms + BathFullCount", data=fsm_df)
fsm_results = fsm.fit()
fsm_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.101
Model:                            OLS   Adj. R-squared:                  0.100
Method:                 Least Squares   F-statistic:                     375.7
Date:                Tue, 29 Sep 2020   Prob (F-statistic):               0.00
Time:                        14:40:22   Log-Likelihood:            -4.6702e+05
No. Observations:               30199   AIC:                         9.341e+05
Df Residuals:                   30189   BIC:                         9.341e+05
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept          -1.309e+06   6.18e+04    -21.175      0.000   -1.43e+06   -1.19e+06
NbrLivingUnits      1.446e+06   4.31e+04     33.525      0.000    1.36e+06    1.53e+06
Stories             1.745e+05   2.29e+04      7.606      0.000     1.3e+05    2.19e+05
SqFt1stFloor          64.3776     26.880      2.395      0.017      11.692     117.063
SqFt2ndFloor        -101.9707     23.247     -4.386      0.000    -147.536     -56.405
SqFtTotLiving        380.9222     18.034     21.122      0.000     345.574     416.270
SqFtGarageAttached    12.1660     32.655      0.373      0.709     -51.839      76.171
SqFtOpenPorch        307.7534     63.810      4.823      0.000     182.683     432.824
Bedrooms           -1.217e+05   9962.601    -12.217      0.000   -1.41e+05   -1.02e+05
BathFullCount      -1.643e+04   1.31e+04     -1.253      0.210   -4.21e+04    9267.502
==============================================================================
Omnibus:                    57055.139   Durbin-Watson:                   0.757
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        152232768.592
Skew:                          14.418   Prob(JB):                         0.00
Kurtosis:                     349.630   Cond. No.                     2.75e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.75e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [18]:
# # create a smaller df to save space and processing power
fsm_df = df_merged[['SalePrice', 'SqFt1stFloor', 'SqFt2ndFloor', 'SqFtTotLiving', 'SqFtGarageAttached','SqFtGarageAttached','SqFtOpenPorch','SqFtEnclosedPorch','Bedrooms','BathHalfCount','BathFullCount']].copy()
fsm_df.dropna(inplace=True)
fsm = ols(formula="SalePrice ~ SqFt1stFloor + SqFt2ndFloor + SqFtTotLiving + SqFtGarageAttached + SqFtGarageAttached + SqFtOpenPorch + SqFtEnclosedPorch + Bedrooms + BathHalfCount + BathFullCount", data=fsm_df)
fsm_results = fsm.fit()
fsm_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.066
Model:                            OLS   Adj. R-squared:                  0.066
Method:                 Least Squares   F-statistic:                     236.9
Date:                Tue, 29 Sep 2020   Prob (F-statistic):               0.00
Time:                        14:40:22   Log-Likelihood:            -4.6759e+05
No. Observations:               30199   AIC:                         9.352e+05
Df Residuals:                   30189   BIC:                         9.353e+05
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept              3.001e+05   3.05e+04      9.826      0.000     2.4e+05     3.6e+05
SqFt1stFloor              6.5141     25.506      0.255      0.798     -43.480      56.508
SqFt2ndFloor            -19.5031     20.158     -0.968      0.333     -59.013      20.007
SqFtTotLiving           371.5526     18.300     20.303      0.000     335.684     407.421
SqFtGarageAttached[0]   -30.1488     16.653     -1.810      0.070     -62.789       2.492
SqFtGarageAttached[1]   -30.1488     16.653     -1.810      0.070     -62.789       2.492
SqFtOpenPorch           287.7428     65.025      4.425      0.000     160.291     415.194
SqFtEnclosedPorch       884.2497    204.802      4.318      0.000     482.830    1285.669
Bedrooms              -8.779e+04   1.01e+04     -8.713      0.000   -1.08e+05    -6.8e+04
BathHalfCount          8502.7352   1.55e+04      0.549      0.583   -2.19e+04    3.89e+04
BathFullCount          3.148e+04   1.33e+04      2.365      0.018    5390.588    5.76e+04
==============================================================================
Omnibus:                    58498.659   Durbin-Watson:                   0.671
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        155297047.098
Skew:                          15.297   Prob(JB):                         0.00
Kurtosis:                     352.976   Cond. No.                     5.25e+16
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 8.95e-23. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [19]:
df

,Merged_Key,DocumentDate,SalePrice,PropertyType,PrincipalUse,PropertyClass,PropType,Area,SubArea,DistrictName,...,SqFtDeck,HeatSystem,Bedrooms,BathHalfCount,Bath3qtrCount,BathFullCount,FpSingleStory,FpMultiStory,YrRenovated,PcntComplete
0,98400000450,2019,409950,11,6,8,R,51.0,6.0,KING COUNTY,...,140.0,5.0,3.0,1.0,0.0,2.0,0.0,1.0,0.0,0.0
1,797320002320,2019,540000,3,6,8,R,23.0,4.0,KING COUNTY,...,0.0,5.0,3.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
2,82607009096,2019,930000,11,6,8,R,70.0,3.0,KING COUNTY,...,0.0,5.0,3.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0
3,410200000075,2019,379950,11,6,8,R,40.0,9.0,KING COUNTY,...,520.0,4.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,868229001120,2019,620000,14,6,8,R,95.0,10.0,KING COUNTY,...,0.0,5.0,3.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3879,147157000030,2019,740000,11,6,8,R,84.0,2.0,KING COUNTY,...,150.0,5.0,3.0,1.0,0.0,2.0,1.0,0.0,0.0,0.0
3880,502940000030,2019,480000,11,6,8,R,23.0,5.0,KING COUNTY,...,0.0,5.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3881,142202009051,2019,522000,11,6,8,R,100.0,5.0,KING COUNTY,...,0.0,4.0,2.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
3882,32606009118,2019,945000,11,6,8,R,95.0,5.0,KING COUNTY,...,0.0,5.0,4.0,0.0,2.0,1.0,0.0,2.0,0.0,0.0


In [20]:
df.select_dtypes(include = 'object').head()

,PropType,DistrictName,WfntAccessRights,WfntProximityInfluence,PowerLines,OtherNuisances,Address,BuildingNumber,ZipCode,DaylightBasement
0,R,KING COUNTY,N,N,N,N,12704 SE 188TH PL 98058,12704,98058,Y
1,R,KING COUNTY,N,N,N,N,10246 5TH AVE SW 98146,10246,98146,
2,R,KING COUNTY,N,N,N,N,30606 NE 183RD ST 98019,30606,98019,
3,R,KING COUNTY,N,N,N,N,37316 WEST LAKE WALKER DR SE 98022,37316,98022,
4,R,KING COUNTY,N,N,N,N,13753 MORGAN DR NE 98053,13753,98053,N


In [21]:
from sklearn.preprocessing import LabelEncoder


In [22]:

def dummying(df, col_name, keep = True):
    if col_name in df.columns:
        label_encoder = LabelEncoder()
        status_labels = label_encoder.fit_transform(df[f"{col_name}"])
        label_encoder.classes_
        df[f"{col_name}_Encoded"] = status_labels
    if keep == False: df.drop(columns = [col_name], inplace = True, errors = 'ignore')
    return df

In [23]:
label_encoder = LabelEncoder()
status_labels = label_encoder.fit_transform(df["WfntAccessRights"])
label_encoder.classes_
df["WfntAccessRights_Encoded"] = status_labels
df.drop(columns = ["WfntAccessRights"], inplace = True, errors = 'ignore')

In [24]:
df["WfntAccessRights_Encoded"]

0       0
1       0
2       0
3       0
4       0
       ..
3879    0
3880    0
3881    0
3882    0
3883    0
Name: WfntAccessRights_Encoded, Length: 3884, dtype: int32

In [25]:
label_encoder = LabelEncoder()
status_labels = label_encoder.fit_transform(df["WfntProximityInfluence"])
label_encoder.classes_
df["WfntProximityInfluence_Encoded"] = status_labels

In [26]:
label_encoder = LabelEncoder()
status_labels = label_encoder.fit_transform(df["PowerLines"])
label_encoder.classes_
df["PowerLines"] = status_labels

In [27]:
label_encoder = LabelEncoder()
status_labels = label_encoder.fit_transform(df["OtherNuisances"])
label_encoder.classes_
df["OtherNuisances_Encoded"] = status_labels

In [28]:
label_encoder = LabelEncoder()
status_labels = label_encoder.fit_transform(df["BuildingNumber"])
label_encoder.classes_
df["BuildingNumber_Encoded"] = status_labels

In [29]:
label_encoder = LabelEncoder()
status_labels = label_encoder.fit_transform(df["ZipCode"])
label_encoder.classes_
df["ZipCode_Encoded"] = status_labels

In [30]:
label_encoder = LabelEncoder()
status_labels = label_encoder.fit_transform(df["DaylightBasement"])
label_encoder.classes_
df["DaylightBasement_Encoded"] = status_labels

In [31]:
df.corr()

,Merged_Key,DocumentDate,SalePrice,PropertyType,PrincipalUse,PropertyClass,Area,SubArea,SqFtLot,WaterSystem,...,FpSingleStory,FpMultiStory,YrRenovated,PcntComplete,WfntAccessRights_Encoded,WfntProximityInfluence_Encoded,OtherNuisances_Encoded,BuildingNumber_Encoded,ZipCode_Encoded,DaylightBasement_Encoded
Merged_Key,1.000000,NaN,-0.022253,0.040551,-0.013608,0.025131,0.006339,0.095817,-0.137731,0.257468,...,0.032982,-0.014389,-0.031693,-0.017287,0.017654,0.007279,0.016402,0.012741,0.038127,-0.005663
DocumentDate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SalePrice,-0.022253,NaN,1.000000,-0.005165,0.015982,-0.060807,0.082551,0.009070,-0.066596,-0.009903,...,0.073894,0.088082,-0.005312,-0.013907,-0.013141,0.000289,0.015179,-0.011847,0.013079,-0.021474
PropertyType,0.040551,NaN,-0.005165,1.000000,-0.018424,-0.060156,-0.050087,0.005188,-0.146624,0.094008,...,0.019104,0.006497,-0.000157,0.332727,0.002993,0.004166,-0.011503,0.017334,-0.000493,0.015197
PrincipalUse,-0.013608,NaN,0.015982,-0.018424,1.000000,-0.333725,-0.016969,0.028120,-0.003165,0.011230,...,0.012974,-0.013239,-0.005288,-0.000785,-0.002618,-0.001264,-0.003965,0.015835,-0.004468,-0.022831
PropertyClass,0.025131,NaN,-0.060807,-0.060156,-0.333725,1.000000,0.001053,0.035990,0.005296,0.007609,...,-0.010030,-0.000211,0.017821,-0.168370,-0.005135,0.002890,0.009069,0.007770,-0.003172,0.012028
Area,0.006339,NaN,0.082551,-0.050087,-0.016969,0.001053,1.000000,0.043277,0.172436,-0.348267,...,0.037990,0.031162,0.058255,0.019493,0.032890,0.009271,0.016722,0.052375,-0.144141,-0.159492
SubArea,0.095817,NaN,0.009070,0.005188,0.028120,0.035990,0.043277,1.000000,-0.051866,0.100274,...,0.064035,0.011829,-0.018884,-0.026043,-0.058128,-0.028200,-0.020166,0.247305,-0.458182,0.018686
SqFtLot,-0.137731,NaN,-0.066596,-0.146624,-0.003165,0.005296,0.172436,-0.051866,1.000000,-0.515045,...,-0.004351,-0.074650,-0.024350,-0.003923,-0.016600,-0.006840,-0.022193,0.005228,-0.051488,-0.133097
WaterSystem,0.257468,NaN,-0.009903,0.094008,0.011230,0.007609,-0.348267,0.100274,-0.515045,1.000000,...,0.087418,0.078535,-0.060439,-0.031428,0.022379,-0.033448,-0.010095,-0.099692,0.152763,0.136653


In [34]:
df.select_dtypes(include = 'object').columns

Index(['PropType', 'DistrictName', 'WfntProximityInfluence', 'OtherNuisances',
       'Address', 'BuildingNumber', 'ZipCode', 'DaylightBasement'],
      dtype='object')

In [33]:
def dummying(df, col_name, keep = True):
    if col_name in df.columns:
        label_encoder = LabelEncoder()
        status_labels = label_encoder.fit_transform(df[f"{col_name}"])
        label_encoder.classes_
        df[f"{col_name}_Encoded"] = status_labels
    if keep == False: df.drop(columns = [col_name], inplace = True, errors = 'ignore')
    return df

In [45]:
def dummying_df(df, keep = True):
    cols = df.select_dtypes(include = 'object').columns
    label_encoder = LabelEncoder()
    for col in cols:
        status_labels = label_encoder.fit_transform(df[f"{col}"])
        df[f"{col}_Encoded"] = status_labels
        if keep == False: df.drop(columns = [col], inplace = True, errors = 'ignore')
    return df

In [46]:
df.select_dtypes(include = 'object').columns

Index(['PropType', 'DistrictName', 'WfntAccessRights',
       'WfntProximityInfluence', 'PowerLines', 'OtherNuisances', 'Address',
       'BuildingNumber', 'ZipCode', 'DaylightBasement'],
      dtype='object')

In [48]:
df = dummying_df(df, False)

In [49]:
df.select_dtypes(include = 'object').columns

Index([], dtype='object')

In [51]:
df

,Merged_Key,DocumentDate,SalePrice,PropertyType,PrincipalUse,PropertyClass,Area,SubArea,SqFtLot,WaterSystem,...,PropType_Encoded,DistrictName_Encoded,WfntAccessRights_Encoded,WfntProximityInfluence_Encoded,PowerLines_Encoded,OtherNuisances_Encoded,Address_Encoded,BuildingNumber_Encoded,ZipCode_Encoded,DaylightBasement_Encoded
0,98400000450,2019,409950,11,6,8,51.0,6.0,7875.0,2.0,...,0,0,0,0,0,0,450,388,22,2
1,797320002320,2019,540000,3,6,8,23.0,4.0,8621.0,2.0,...,0,0,0,0,0,0,63,53,34,0
2,82607009096,2019,930000,11,6,8,70.0,3.0,212911.0,1.0,...,0,0,0,0,0,0,2564,2206,6,0
3,410200000075,2019,379950,11,6,8,40.0,9.0,14149.0,1.0,...,0,0,0,0,0,0,2884,2507,7,0
4,868229001120,2019,620000,14,6,8,95.0,10.0,4046.0,2.0,...,0,0,0,0,0,0,645,548,20,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3879,147157000030,2019,740000,11,6,8,84.0,2.0,72465.0,2.0,...,0,0,0,0,0,0,2167,1844,13,2
3880,502940000030,2019,480000,11,6,8,23.0,5.0,7500.0,2.0,...,0,0,0,0,0,0,167,138,34,0
3881,142202009051,2019,522000,11,6,8,100.0,5.0,201682.0,1.0,...,0,0,0,0,0,0,2052,1736,26,0
3882,32606009118,2019,945000,11,6,8,95.0,5.0,165528.0,1.0,...,0,0,0,0,0,0,1639,1364,29,0
